In [1]:
import pandas as pd
import numpy as np
import regex as re
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("..")
from src.clean import *  #funciones


In [3]:
actor = "../data/actor.csv"
a = abrir_csv(actor)
category = "../data/category.csv"
c = abrir_csv(category)
film = "../data/film.csv"
f = abrir_csv(film)
inventory = "../data/inventory.csv"
i = abrir_csv(inventory)
language = "../data/language.csv"
l = abrir_csv(language)
old_HDD = "../data/old_HDD.csv"
o = abrir_csv(old_HDD)
rental = "../data/rental.csv"
r = abrir_csv(rental)


# Exploración archivos y limpieza previa.

### 1º) Actores.

In [4]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [5]:
a.describe()

,actor_id
count,200.000000
mean,100.500000
std,57.879185
min,1.000000
25%,50.750000
50%,100.500000
75%,150.250000
max,200.000000


In [6]:
a.columns=titulos(a) #ponemos bien los títulos
verificar_nulos(a) #No hay ningun valor nulo

,Valores Nulos,Valores Nan
Actor Id,0,0
First Name,0,0
Last Name,0,0
Last Update,0,0


In [7]:
a.columns

Index(['Actor Id', 'First Name', 'Last Name', 'Last Update'], dtype='object')

In [8]:
convertir_fecha(a)
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Actor Id     200 non-null    int64         
 1   First Name   200 non-null    object        
 2   Last Name    200 non-null    object        
 3   Last Update  200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 6.4+ KB


In [9]:
a[a.duplicated(subset=['First Name', 'Last Name'],keep=False)] # busco duplicados pero me centro solo en los nombes ya que los id son diferentes para cada uno,
                                                               # hay un actor repetido, vamos a quitarlo. El keep False lo ponggo porque si no
                                                               #solo me mostraba el 109 con el nombre y no sabia de donde venía el duplicado.

,Actor Id,First Name,Last Name,Last Update
100,101,SUSAN,DAVIS,2006-02-15 04:34:33
109,110,SUSAN,DAVIS,2006-02-15 04:34:33


In [10]:
a.drop([109],axis=0,inplace=True) # no me va a interferir porque no hay ninguna tabla que me relacione su id con nada, no peta 💣

In [11]:
a.reset_index(inplace=True,drop=True)
nuevos_id = pd.Series(range(1, len(a)+1)) # cambio los id para que no tener salto y reseteo index
a['Actor Id']=nuevos_id

a.iloc[107:112] # compruebo que no me pegue el salto, he hecho todo esto porque para el análisis me puede servir

,Actor Id,First Name,Last Name,Last Update
107,108,WARREN,NOLTE,2006-02-15 04:34:33
108,109,SYLVESTER,DERN,2006-02-15 04:34:33
109,110,CAMERON,ZELLWEGER,2006-02-15 04:34:33
110,111,RUSSELL,BACALL,2006-02-15 04:34:33
111,112,MORGAN,HOPKINS,2006-02-15 04:34:33


### 2º)Category:

In [12]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category_id  16 non-null     int64 
 1   name         16 non-null     object
 2   last_update  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 512.0+ bytes


In [13]:
c.head()

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [14]:
c.describe().T

,count,mean,std,min,25%,50%,75%,max
category_id,16.0,8.5,4.760952,1.0,4.75,8.5,12.25,16.0


In [15]:
c.columns=titulos(c)
convertir_fecha(c)
verificar_nulos(c)

,Valores Nulos,Valores Nan
Category Id,0,0
Name,0,0
Last Update,0,0


In [16]:
duplicados(c)

0

### 3º) Film:

In [17]:
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  0 non-null      float64
 6   rental_duration       1000 non-null   int64  
 7   rental_rate           1000 non-null   float64
 8   length                1000 non-null   int64  
 9   replacement_cost      1000 non-null   float64
 10  rating                1000 non-null   object 
 11  special_features      1000 non-null   object 
 12  last_update           1000 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 101.7+ KB


In [18]:
f.describe().T

,count,mean,std,min,25%,50%,75%,max
film_id,1000.0,500.500,288.819436,1.00,250.75,500.50,750.25,1000.00
release_year,1000.0,2006.000,0.000000,2006.00,2006.00,2006.00,2006.00,2006.00
language_id,1000.0,1.000,0.000000,1.00,1.00,1.00,1.00,1.00
original_language_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rental_duration,1000.0,4.985,1.411654,3.00,4.00,5.00,6.00,7.00
rental_rate,1000.0,2.980,1.646393,0.99,0.99,2.99,4.99,4.99
length,1000.0,115.272,40.426332,46.00,80.00,114.00,149.25,185.00
replacement_cost,1000.0,19.984,6.050833,9.99,14.99,19.99,24.99,29.99


In [19]:
f.columns=titulos(f)
convertir_fecha(f)
verificar_nulos(f)

,Valores Nulos,Valores Nan
Film Id,0,0
Title,0,0
Description,0,0
Release Year,0,0
Language Id,0,0
Original Language Id,1000,1000
Rental Duration,0,0
Rental Rate,0,0
Length,0,0
Replacement Cost,0,0


In [20]:
f.drop(columns=['Original Language Id'], inplace=True)  #tiene una columna llena de nulos, la quitamos✂️

Rellenamos la columna de Language Id con numeros aleatorios para luego relacionarla con language:


In [21]:

f['Language Id'] = [random.randint(1, 6) for i in range(len(f))]

Borrarmos la columna de Release Year, ya que las fechas son incorrectas al ser mayores a la fechas de alquiler


In [22]:
f = f.drop('Release Year', axis=1)

### 4º) Inventory:

In [23]:
i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   inventory_id  1000 non-null   int64 
 1   film_id       1000 non-null   int64 
 2   store_id      1000 non-null   int64 
 3   last_update   1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [24]:
i.describe().T

,count,mean,std,min,25%,50%,75%,max
inventory_id,1000.0,500.500,288.819436,1.0,250.75,500.5,750.25,1000.0
film_id,1000.0,109.866,63.862042,1.0,56.00,111.5,164.00,223.0
store_id,1000.0,1.497,0.500241,1.0,1.00,1.0,2.00,2.0


In [25]:
i.columns=titulos(i)
convertir_fecha(i)
verificar_nulos(i)

,Valores Nulos,Valores Nan
Inventory Id,0,0
Film Id,0,0
Store Id,0,0
Last Update,0,0


### 5º) Language:

In [26]:
l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   language_id  6 non-null      int64 
 1   name         6 non-null      object
 2   last_update  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 272.0+ bytes


In [27]:
l.describe().T

,count,mean,std,min,25%,50%,75%,max
language_id,6.0,3.5,1.870829,1.0,2.25,3.5,4.75,6.0


In [28]:
l.columns=titulos(l)
convertir_fecha(l)
verificar_nulos(l)

,Valores Nulos,Valores Nan
Language Id,0,0
Name,0,0
Last Update,0,0


### 6º) old_HDD:

In [29]:
o.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [30]:
o.describe().T

,count,mean,std,min,25%,50%,75%,max
release_year,1000.0,2006.000,0.000000,2006.0,2006.0,2006.0,2006.0,2006.0
category_id,1000.0,8.355,4.726872,1.0,4.0,8.0,13.0,16.0


In [31]:
o.columns=titulos(o)
verificar_nulos(o)

,Valores Nulos,Valores Nan
First Name,0,0
Last Name,0,0
Title,0,0
Release Year,0,0
Category Id,0,0


In [32]:
o.columns

Index(['First Name', 'Last Name', 'Title', 'Release Year', 'Category Id'], dtype='object')

In [33]:
o[o.duplicated(subset=['Title', 'First Name', 'Last Name'], keep=False)] # ningun duplicado, los he pasado así por si había algun actor repetido dentro de una 
                                                                         # misma peli


,First Name,Last Name,Title,Release Year,Category Id


### Vamos a asignarles el mismo id que tenemos en la tabla de actors para luego hacer las relaciones:

In [34]:
o = o.merge(a[['Actor Id','First Name', 'Last Name']], on=['First Name', 'Last Name'], how='left')

### Tambien voy a asignar el id de la película

In [35]:
o = o.merge(f[['Film Id','Title']], on=['Title'], how='left')

In [36]:
o.head()

,First Name,Last Name,Title,Release Year,Category Id,Actor Id,Film Id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6,1,1
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2,1,23
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13,1,25
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10,1,106
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14,1,140


## 6º) Rentail 

In [37]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     1000 non-null   int64 
 1   rental_date   1000 non-null   object
 2   inventory_id  1000 non-null   int64 
 3   customer_id   1000 non-null   int64 
 4   return_date   1000 non-null   object
 5   staff_id      1000 non-null   int64 
 6   last_update   1000 non-null   object
dtypes: int64(4), object(3)
memory usage: 54.8+ KB


In [38]:
r['rental_date'] = pd.to_datetime(r['rental_date'], format="%Y-%m-%d %H:%M:%S") #paso rental_date y return_date a dataTime
r['return_date'] = pd.to_datetime(r['return_date'], format="%Y-%m-%d %H:%M:%S")

In [39]:
r.describe().T

,count,mean,min,25%,50%,75%,max,std
rental_id,1000.0,501.18,1.0,250.75,501.5,751.25,1001.0,289.19665
rental_date,1000,2005-05-28 00:43:08.471999872,2005-05-24 22:53:30,2005-05-26 14:34:21,2005-05-28 01:22:09.500000,2005-05-29 10:00:21,2005-05-31 00:46:31,NaN
inventory_id,1000.0,2258.179,2.0,1157.5,2244.5,3371.75,4581.0,1314.667454
customer_id,1000.0,296.408,1.0,150.0,296.0,445.25,597.0,172.509319
return_date,1000,2005-06-01 22:59:55.512000128,2005-05-25 23:55:21,2005-05-30 12:40:22.750000,2005-06-01 22:56:22.500000,2005-06-04 09:36:58.750000,2005-06-09 01:19:28,NaN
staff_id,1000.0,1.521,1.0,1.0,2.0,2.0,2.0,0.499809


In [40]:
r.columns=titulos(r)
convertir_fecha(r)
verificar_nulos(r)

,Valores Nulos,Valores Nan
Rental Id,0,0
Rental Date,0,0
Inventory Id,0,0
Customer Id,0,0
Return Date,0,0
Staff Id,0,0
Last Update,0,0


In [41]:
check = (r['Rental Date'] < r['Return Date']).all()  #reviso que las fechas de devolución sean mayores a las adquisición.
check

True

## Exportamos archivos limpios.

Una vez tenemos lo dataframe limpios procedemos a optimizar los datos

In [42]:
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Film Id           1000 non-null   int64         
 1   Title             1000 non-null   object        
 2   Description       1000 non-null   object        
 3   Language Id       1000 non-null   int64         
 4   Rental Duration   1000 non-null   int64         
 5   Rental Rate       1000 non-null   float64       
 6   Length            1000 non-null   int64         
 7   Replacement Cost  1000 non-null   float64       
 8   Rating            1000 non-null   object        
 9   Special Features  1000 non-null   object        
 10  Last Update       1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 86.1+ KB


In [43]:
a=optim(a)
c=optim(c)
f=optim(f)
i=optim(i)
l=optim(l)
o=optim(o)
r=optim(r)

In [44]:
f.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Film Id           1000 non-null   int16         
 1   Title             1000 non-null   category      
 2   Description       1000 non-null   category      
 3   Language Id       1000 non-null   int8          
 4   Rental Duration   1000 non-null   int8          
 5   Rental Rate       1000 non-null   float32       
 6   Length            1000 non-null   int16         
 7   Replacement Cost  1000 non-null   float32       
 8   Rating            1000 non-null   category      
 9   Special Features  1000 non-null   category      
 10  Last Update       1000 non-null   datetime64[ns]
dtypes: category(4), datetime64[ns](1), float32(2), int16(2), int8(2)
memory usage: 311.2 KB


In [45]:
actor_limpio =a

category_limpio = c

film_limpio =f

inventory_limpio = i

language_limpio = l

old_HDD_limpio = o

rental_limpio = r 

In [46]:
save(actor_limpio,"../data/actor_limpio.csv")
save(category_limpio,"../data/category_limpio.csv")
save(film_limpio,"../data/film_limpio.csv")
save(inventory_limpio,"../data/inventory_limpio.csv")
save(language_limpio,"../data/language_limpio.csv")
save(old_HDD_limpio,"../data/old_HDD_limpio.csv")
save(rental_limpio,"../data/rental_limpio.csv") #Al finla decido no subirlo pero dejo el código para futuros

# FIN